# 1. 상관관계

In [122]:
# 데이터
import pandas as pd
df = pd.DataFrame({
    '키': [150, 160, 170, 175, 165],
    '몸무게': [42, 52, 75, 67, 56]
})

In [123]:
# 상관계수
df.corr()

,키,몸무게
키,1.000000,0.901012
몸무게,0.901012,1.000000


In [124]:
df['키'].corr(df['몸무게'])

0.901011928876437

In [125]:
# 피어슨
print(df.corr())
# 스피어맨
print(df.corr(method='spearman'))
# 켄달타우
print(df.corr(method='kendall'))

            키       몸무게
키    1.000000  0.901012
몸무게  0.901012  1.000000
       키  몸무게
키    1.0  0.9
몸무게  0.9  1.0
       키  몸무게
키    1.0  0.8
몸무게  0.8  1.0


In [126]:
# t검정
from scipy import stats
# 피어슨
print(stats.pearsonr(df['키'], df['몸무게']))

# 스피어맨
print(stats.spearmanr(df['키'], df['몸무게']))

# 켄달타우
print(stats.kendalltau(df['키'], df['몸무게']))

PearsonRResult(statistic=0.9010119288764369, pvalue=0.03682580945087895)
SignificanceResult(statistic=0.8999999999999998, pvalue=0.03738607346849874)
SignificanceResult(statistic=0.7999999999999999, pvalue=0.08333333333333333)


# 단순 선형 회귀 분석

주어진 키와 몸무게 데이터로 회귀모델을 구축하고 각 소문제의 값을 구하시오.
- 키: 종속변수
- 몸무게: 독립변수

In [127]:
import pandas as pd

df = pd.DataFrame({
    '키': [150, 160, 170, 175, 165, 155, 172, 168, 174, 158,
          162, 173, 156, 159, 167, 163, 171, 169, 176, 161],
    '몸무게': [74, 50, 70, 64, 56, 48, 68, 60, 65, 52,
            54, 67, 49, 51, 58, 55, 69, 61, 66, 53]
})

In [128]:
# 모델 학습 summary 출력
from statsmodels.formula.api import ols
model = ols("키 ~ 몸무게", data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      키   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     6.984
Date:                Fri, 29 Nov 2024   Prob (F-statistic):             0.0165
Time:                        09:05:43   Log-Likelihood:                -64.701
No. Observations:                  20   AIC:                             133.4
Df Residuals:                      18   BIC:                             135.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    135.8209     11.211     12.115      0.0

In [129]:
# 결정계수
# 0.280
model.rsquared

0.27954323113299584

In [130]:
# 기울기(회귀계수)
# 0.4938
model.params['몸무게']

0.49376558603491283

In [131]:
# 절편(회귀계수)
# 135.8209
model.params['Intercept']

135.8209476309227

In [132]:
# 몸무게의 회귀계수가 통계적으로 유의한지 pvalue
# 0.017
model.pvalues['몸무게']

0.01654013445316978

In [133]:
# 몸무게가 50 일때 예측키
new_data = pd.DataFrame({
    '몸무게':[50]
})
model.predict(new_data)
# 160.509227

,0
0,160.509227


In [134]:
# 잔차 제곱합
# 잔차 = 관측(실제)값 - 예측값
df['잔차'] = df['키'] - model.predict(df['몸무게'])
df['잔차제곱'] = (df['키'] - model.predict(df['몸무게'])) ** 2
sum(df['잔차제곱'])

755.9032418952607

In [135]:
# MSE
df['잔차제곱'].mean()

37.79516209476303

In [136]:
# 사이킷런 MSE
from sklearn.metrics import mean_squared_error
pred = model.predict(df)
mean_squared_error(df['키'], pred)

37.79516209476303

In [137]:
# 신뢰구간
# 0.101       0.886

In [138]:
# 몸무게가 50일 때 예측키에 대한 신뢰구간, 예측구간
new_data = pd.DataFrame({
    '몸무게':[50]
})
pred = model.get_prediction(new_data)
pred.summary_frame()

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,160.509227,2.291332,155.695318,165.323136,146.068566,174.949888


In [139]:
# 신뢰구간: 155.695318	165.323136
# 예측구간: 146.068566	174.949888

# 다중 선형 회귀 분석

주어진 매출액, 광고비, 플랫폼 데이터로 회귀모델을 구축하고 각 소문제의 값을 구하시오.
- 매출액: 종속변수
- 광고비, 플랫폼(유통 플랫폼 수), 투자: 독립변수

In [140]:
import pandas as pd
df = pd.DataFrame({
    '매출액': [300, 320, 250, 360, 315, 328, 310, 335, 326, 280,
            290, 300, 315, 328, 310, 335, 300, 400, 500, 600],
    '광고비': [70, 75, 30, 80, 72, 77, 70, 82, 70, 80,
            68, 90, 72, 77, 70, 82, 40, 20, 75, 80],
    '플랫폼': [15, 16, 14, 20, 19, 17, 16, 19, 15, 20,
            14, 5, 16, 17, 16, 14, 30, 40, 10, 50],
    '투자':[100, 0, 200, 0, 10, 0, 5, 0, 20, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    })
df.head(3)

,매출액,광고비,플랫폼,투자
0,300,70,15,100
1,320,75,16,0
2,250,30,14,200


In [141]:
# 모델 학습 summary 출력 (종속변수: 매출액, 독립변수: 광고비, 플랫폼)
from statsmodels.formula.api import ols
formula = "매출액~광고비+플랫폼"
model = ols(formula, data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    매출액   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     8.907
Date:                Fri, 29 Nov 2024   Prob (F-statistic):            0.00226
Time:                        09:05:44   Log-Likelihood:                -108.22
No. Observations:                  20   AIC:                             222.4
Df Residuals:                      17   BIC:                             225.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    101.0239     71.716      1.409      0.1

In [142]:
# 결정계수(R-squared)
# 0.512
model.rsquared

0.5116964327009041

In [143]:
# 회귀계수(광고비)
# 1.8194
model.params['광고비']

1.8194269419344642

In [144]:
# 회귀계수(플랫폼)
# 5.9288
model.params['플랫폼']

5.928755546950749

In [145]:
# 회귀계수(절편)
# 101.0239
model.params['Intercept']

101.02387228241548

In [146]:
# 광고비의 회귀계수가 통계적으로 유의한지 pvalue
# 0.038
model.pvalues['광고비']

0.03764350647695994

In [147]:
# 플랫폼의 회귀계수가 통계적으로 유의한지 pvalue
# 0.001
model.pvalues['플랫폼']

0.0006746810554564365

In [148]:
# 광고비 50, 플랫폼 20일 때 매출액 예측
new_data = pd.DataFrame({
    '광고비':[50],
    '플랫폼':[20]
})
model.predict(new_data)
# 310.57033

,0
0,310.57033


In [149]:
# 잔차 제곱합
sum(model.resid ** 2)

58686.178271561075

In [150]:
# MSE
(model.resid ** 2).mean()

2934.3089135780533

In [151]:
# 광고비, 플랫폼 회귀계수의 95% 신뢰구간
# 광고비: 0.117       3.522
# 플랫폼: 2.912       8.945

In [152]:
# 광고비 50, 플랫폼 20일 때 매출액에 대한 95% 신뢰구간과 예측구간
# 광고비 50, 플랫폼 20일 때 매출액 예측
new_data = pd.DataFrame({
    '광고비':[50],
    '플랫폼':[20]
})
pred = model.get_prediction(new_data)
pred.summary_frame()
# 신뢰구간: 268.612221	352.52844
# 예측구간: 179.700104	441.440556

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,310.57033,19.887098,268.612221,352.52844,179.700104,441.440556


### 범주형 변수

In [153]:
import pandas as pd
df = pd.DataFrame({
    '매출액': [300, 320, 250, 360, 315, 328, 310, 335, 326, 280,
            290, 300, 315, 328, 310, 335, 300, 400, 500, 600],
    '광고비': [70, 75, 30, 80, 72, 77, 70, 82, 70, 80,
            68, 90, 72, 77, 70, 82, 40, 20, 75, 80],
    '플랫폼': [15, 16, 14, 20, 19, 17, 16, 19, 15, 20,
            14, 5, 16, 17, 16, 14, 30, 40, 10, 50],
    '투자':[100, 0, 200, 0, 10, 0, 5, 0, 20, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '유형':['B','B','C','A','B','B','B','B','B','B'
        ,'C','B','B','B','B','B','B','A','A','A']
    })
df.head(3)

,매출액,광고비,플랫폼,투자,유형
0,300,70,15,100,B
1,320,75,16,0,B
2,250,30,14,200,C


In [154]:
from statsmodels.formula.api import ols
formula = "매출액~광고비+유형"
model = ols(formula, data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    매출액   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     13.70
Date:                Fri, 29 Nov 2024   Prob (F-statistic):           0.000110
Time:                        09:05:44   Log-Likelihood:                -102.67
No. Observations:                  20   AIC:                             213.3
Df Residuals:                      16   BIC:                             217.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    400.6463     47.477      8.439      0.0

In [155]:
# 원핫인코딩
df2 = pd.get_dummies(df)

In [156]:
# drop_first=True
df2 = pd.get_dummies(df, drop_first=True)

In [157]:
# 회귀 모델 학습 & summary (독립변수로 광고비와 유형만 활용)
formula = "매출액~광고비+유형_B+유형_C"
model = ols(formula, data=df2).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    매출액   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     13.70
Date:                Fri, 29 Nov 2024   Prob (F-statistic):           0.000110
Time:                        09:05:44   Log-Likelihood:                -102.67
No. Observations:                  20   AIC:                             213.3
Df Residuals:                      16   BIC:                             217.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      400.6463     47.477      8.439   